In [1]:
import numpy as np
import cv2


import mediapipe as mp

import PoseTrackingModule as ptm
import os

from yoloV3 import YOLOdetector

from DepthDetector import DepthDetector
np.set_printoptions(threshold=sys.maxsize)

In [2]:
main_folder_name="Recordings/mixes_copy/"

Spark_filename= main_folder_name+"Spark_image/"
Spark_numpys=main_folder_name+"Spark_npys/"
Spark_mins=main_folder_name+"Spark_mins/"
Spark_maxs=main_folder_name+"Spark_maxs/"

Color_filename= main_folder_name+"Color_RealSense/"
Depth_filename= main_folder_name+"Depth_RealSense/"
Depthnp_filename = main_folder_name+"Depthnp_RealSense/"

file_ext = ".jpg"


In [3]:
Color_RealSense= []
Depth_RealSense= []
Depthnp_RealSense= []
Spark_image= []
Spark_maxS= []
Spark_minS= []
Spark_npys= []


for file in os.listdir(Color_filename):
    Color_RealSense.append(cv2.imread(Color_filename+file))

for file in os.listdir(Depth_filename):
    Depth_RealSense.append(cv2.imread(Depth_filename+file))

for file in os.listdir(Depthnp_filename):
    Depthnp_RealSense.append(np.load(Depthnp_filename+file))

for file in os.listdir(Spark_filename):
    Spark_image.append(cv2.imread(Spark_filename+file))

for file in os.listdir(Spark_maxs):
    Spark_maxS.append(np.load(Spark_maxs+file))

for file in os.listdir(Spark_mins):
    Spark_minS.append(np.load(Spark_mins+file))

for file in os.listdir(Spark_numpys):
    Spark_npys.append(np.load(Spark_numpys+file))


depth_scale=np.load(main_folder_name+'depth_scale.npy')

In [4]:
detector = ptm.poseDetector()
depthDetector=DepthDetector(depth_scale=depth_scale)
yolo= YOLOdetector()

In [5]:
for i in range(len(Color_RealSense)):
    grideye_image = Spark_image[i]
    color_image  = Color_RealSense[i]
    depth_colormap = Depth_RealSense[i]
    grideye_values=Spark_npys[i]
    minTemp=Spark_minS[i]
    maxTemp=Spark_maxS[i]
    depth_map=Depthnp_RealSense[i]
    yoloDetects = yolo.predict(color_image)
    landmarks=[]
    for yoloDetect in yoloDetects:
        detectPoseRGB, detectPoseDepth, landmark = detector.findPoseAndDrawLandmarks(
        color_image[yoloDetect[1]:yoloDetect[3],yoloDetect[0]:yoloDetect[2]], depth_map[yoloDetect[1]:yoloDetect[3],yoloDetect[0]:yoloDetect[2]])
        landmarks.append(landmark)
    
    for l in landmarks:
        if(l):
            depth_map=depthDetector.detect(l,depth_map,grideye_values,minTemp,maxTemp)
    
    
    cv2.imshow("Color RealSense image", color_image)
    print()
    print()
    print()
    print()
    #cv2.imshow("Depth RealSense image", detectPoseDepth)
    #cv2.imshow("Dete RealSense image", detectPoseRGB)
    k = cv2.waitKey(0)
    if k == 27 or i==len(Color_RealSense)-1:
        cv2.destroyAllWindows()
        break


488219
FACE flat_factor 332.65855742774545
ThermalConfidence 1
DepthConfidence 3
125474
FACE flat_factor 1116.307261320587
ThermalConfidence 1
DepthConfidence 3
4028814
FACE flat_factor 198230.7853533433
ThermalConfidence 1
DepthConfidence 3




136761
FACE flat_factor 28342.119867440808
ThermalConfidence 1
DepthConfidence 3






##mediapipe only
for i in range(len(Color_RealSense)):
    grideye_image = Spark_image[i]
    color_image  = Color_RealSense[i]
    depth_colormap = Depth_RealSense[i]
    grideye_values=Spark_npys[i]
    minTemp=Spark_minS[i]
    maxTemp=Spark_maxS[i]
    depth_map=Depthnp_RealSense[i]



    detectPoseRGB, detectPoseDepth, landmarks = detector.findPoseAndDrawLandmarks(
    color_image, depth_map)

    
    if(landmarks):
        depth_map=depthDetector.detect(landmarks,depth_map,grideye_values,minTemp,maxTemp)
    
    
    # cv2.imshow("Color RealSense image", color_image)
    print()
    print()
    print()
    print()
    #cv2.imshow("Depth RealSense image", detectPoseDepth)
    #cv2.imshow("Dete RealSense image", detectPoseRGB)
    k = 0#cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

In [ ]:
points=[]

depths=[]
#get the depths inside the four points inside which the object is
for i in range(int(landmarks[24][0]),int(landmarks[23][0])):
    for j in range (int(landmarks[12][1]),int(landmarks[24][1])):
            depths.append([float(depth_map[j][i].astype(float)*self.depth_scale*1000)])#distance in cm
depths = np.array(depths)
points = np.array(points)
# Fit a plane to the points using the RANSAC algorithm
ransac = RANSACRegressor(min_samples=3)
model = ransac.fit(points, depths)